## Importer Pakker

In [139]:
import pandas as pd

In [140]:
transaksjoner = pd.read_csv("../data/utfyllende_transaksjon.csv")

## Dataformat

La oss sjekke dataformatet på kolonnene

In [141]:
transaksjoner.dtypes

id                int64
userId            int64
stockId           int64
groupId           int64
semesterId        int64
logTimestamp     object
price           float64
stockName        object
volume          float64
groupName        object
dtype: object

In [142]:
transaksjoner.logTimestamp = pd.to_datetime(transaksjoner.logTimestamp)
transaksjoner.head(2)

,id,userId,stockId,groupId,semesterId,logTimestamp,price,stockName,volume,groupName
0,2,6,7,1,1,2022-03-23 17:08:52,21.0,"Coca Cola 0,5",0.5,it.gruppen
1,5,5,7,1,1,2022-03-23 17:33:12,21.0,"Coca Cola 0,5",0.5,it.gruppen


In [143]:
transaksjoner.dtypes

id                       int64
userId                   int64
stockId                  int64
groupId                  int64
semesterId               int64
logTimestamp    datetime64[ns]
price                  float64
stockName               object
volume                 float64
groupName               object
dtype: object

### Hvor mange ulike verdier har vi for hver av kolonnene

In [144]:
transaksjoner.nunique()

id              16706
userId            560
stockId            80
groupId            24
semesterId          3
logTimestamp    15052
price              37
stockName          64
volume              9
groupName          24
dtype: int64

### Litt rart at semesterId har 3 unike verdier, er det ikke?

In [145]:
transaksjoner.loc[transaksjoner.semesterId == 1].logTimestamp.min()

Timestamp('2022-03-23 17:08:52')

In [146]:
first_semester_id_min = transaksjoner.loc[transaksjoner.semesterId == 1].logTimestamp.min()
first_semester_id_max = transaksjoner.loc[transaksjoner.semesterId == 1].logTimestamp.max()
print(f"semesterId 1: min: {first_semester_id_min}")
print(f"semesterId 1: max: {first_semester_id_max}")

second_semester_id_min = transaksjoner.loc[transaksjoner.semesterId == 2].logTimestamp.min()
second_semester_id_max = transaksjoner.loc[transaksjoner.semesterId == 2].logTimestamp.max()
print(f"semesterId 2: min: {second_semester_id_min}")
print(f"semesterId 2: max: {second_semester_id_max}")

third_semester_id_min = transaksjoner.loc[transaksjoner.semesterId == 3].logTimestamp.min()
third_semester_id_max = transaksjoner.loc[transaksjoner.semesterId == 3].logTimestamp.max()
print(f"semesterId 3: min: {third_semester_id_min}")
print(f"semesterId 3: max: {third_semester_id_max}")

semesterId 1: min: 2022-03-23 17:08:52
semesterId 1: max: 2022-07-30 19:55:38
semesterId 2: min: 2022-08-04 17:52:04
semesterId 2: max: 2022-12-30 15:15:07
semesterId 3: min: 2023-01-01 22:30:17
semesterId 3: max: 2023-02-19 12:23:18


In [147]:
transaksjoner.groupby(["semesterId"]).logTimestamp.agg(["min", "max"])

,min,max
semesterId,,
1,2022-03-23 17:08:52,2022-07-30 19:55:38
2,2022-08-04 17:52:04,2022-12-30 15:15:07
3,2023-01-01 22:30:17,2023-02-19 12:23:18


## Semesterkolonne
Her mangler det informasjon om det er høst/vår semesteret

In [148]:
def which_semester(month: int) -> str:
    if month <= 6:
        return "vår"
    return "høst"

transaksjoner["mnd"] = transaksjoner.logTimestamp.dt.month
transaksjoner["semester"] = transaksjoner.mnd.apply(which_semester)
transaksjoner.head(2)

,id,userId,stockId,groupId,semesterId,logTimestamp,price,stockName,volume,groupName,mnd,semester
0,2,6,7,1,1,2022-03-23 17:08:52,21.0,"Coca Cola 0,5",0.5,it.gruppen,3,vår
1,5,5,7,1,1,2022-03-23 17:33:12,21.0,"Coca Cola 0,5",0.5,it.gruppen,3,vår


## Legge til kolonner fra eksisterende kolonner

In [ ]:
transaksjoner["mva"] = transaksjoner.price * 0.25

In [ ]:
transaksjoner["total_pris"] = transaksjoner.price + transaksjoner.mva

In [ ]:
markup = 0.1
markupfaktor = 1 + markup
transaksjoner["kost_pris"] = transaksjoner.price / markupfaktor

## Hvor mange observasjoner har vi av hvert produkt

In [149]:
transaksjoner.stockName.value_counts()

Red Bull              4515
Olden Eple            1289
Red Bull Sugarfree    1200
Brus 0,3              1113
Brus                   989
                      ... 
Sprite 0,33              5
Red Bull 0,3             5
Lollipop                 3
Burn utgått              2
Burn 0,33                1
Name: stockName, Length: 64, dtype: int64

Avhengig av hva vi er ute etter kan det kanskje være hensiktsmessig å gruppere disse litt mer

- F.eks. brus 0.5 og brus
- Manuelt arbeid, men det er det ofte når man jobber med datakvalitet

In [61]:
stock_list = transaksjoner.stockName.unique().tolist()
stock_dict = dict(zip(stock_list, stock_list))
stock_dict

{'Coca Cola 0,5': 'Coca Cola 0,5',
 'Red Bull': 'Red Bull',
 'Julebrus': 'Julebrus',
 'Eplemost': 'Eplemost',
 'Tikronersorama': 'Tikronersorama',
 'Olden Eple': 'Olden Eple',
 'Coca Cola Zero 0,33': 'Coca Cola Zero 0,33',
 'Red Bull Yellow ': 'Red Bull Yellow ',
 'Red Bull Sugarfree': 'Red Bull Sugarfree',
 'Coca Cola Zero 0,5': 'Coca Cola Zero 0,5',
 'Coca Cola 0,33': 'Coca Cola 0,33',
 'Burn 0,33': 'Burn 0,33',
 'Monster White Ultra': 'Monster White Ultra',
 'Brus': 'Brus',
 'Brus 0,5': 'Brus 0,5',
 'Sjokolade': 'Sjokolade',
 'Aass UTEN, Sitron og lime': 'Aass UTEN, Sitron og lime',
 'Aass UTEN': 'Aass UTEN',
 'Pizza': 'Pizza',
 'Bandgenser': 'Bandgenser',
 'Red Bull 0,3': 'Red Bull 0,3',
 'Nudler': 'Nudler',
 'Billett på kryss': 'Billett på kryss',
 'Billys Pan Pizza': 'Billys Pan Pizza',
 'Appelsin Juice': 'Appelsin Juice',
 'Brus 0,3': 'Brus 0,3',
 'Fanta': 'Fanta',
 'Sprite 0,33': 'Sprite 0,33',
 'Olden med kullsyre': 'Olden med kullsyre',
 'Monster hvit 0,5': 'Monster hvit 0,5'

In [62]:
updated_stock_dict = {'Coca Cola 0,5': 'Coca Cola 0,5',
 'Red Bull': 'Red Bull',
 'Julebrus': 'Julebrus',
 'Eplemost': 'Eplemost',
 'Tikronersorama': 'Tikronersorama',
 'Olden Eple': 'Olden Eple',
 'Coca Cola Zero 0,33': 'Coca Cola Zero 0,33',
 'Red Bull Yellow ': 'Red Bull Yellow ',
 'Red Bull Sugarfree': 'Red Bull Sugarfree',
 'Coca Cola Zero 0,5': 'Coca Cola Zero 0,5',
 'Coca Cola 0,33': 'Coca Cola 0,33',
 'Burn 0,33': 'Burn 0,33',
 'Monster White Ultra': 'Monster White Ultra',
 'Brus': 'Brus 0,5',
 'Brus 0,5': 'Brus 0,5',
 'Sjokolade': 'Sjokolade',
 'Aass UTEN, Sitron og lime': 'Aass UTEN, Sitron og lime',
 'Aass UTEN': 'Aass UTEN',
 'Pizza': 'Pizza',
 'Bandgenser': 'Bandgenser',
 'Red Bull 0,3': 'Red Bull 0,3',
 'Nudler': 'Nudler',
 'Billett på kryss': 'Billett på kryss',
 'Billys Pan Pizza': 'Billys Pan Pizza',
 'Appelsin Juice': 'Appelsin Juice',
 'Brus 0,3': 'Brus 0,3',
 'Fanta': 'Fanta',
 'Sprite 0,33': 'Sprite 0,33',
 'Olden med kullsyre': 'Olden med kullsyre',
 'Monster hvit 0,5': 'Monster hvit 0,5',
 'Kaffekapsel': 'Kaffekapsel',
 'Coca Cola Glass': 'Coca Cola Glass',
 'Coca Cola Zero Glass': 'Coca Cola Zero Glass',
 'Utgått vare': 'Utgått vare',
 'Sprite 0,5': 'Sprite 0,5',
 'Sol 0,33': 'Sol 0,33',
 'Farris Bris': 'Farris Bris',
 'Burn 0,5': 'Burn 0,5',
 'Proteinshake | Scoop': 'Proteinshake | Scoop',
 'Powerking': 'Powerking',
 'Olden': 'Olden',
 'Sprite': 'Sprite',
 'Cola': 'Cola',
 'Cola Zero': 'Cola Zero',
 'Julebrus Hansa': 'Julebrus Hansa',
 'Burn': 'Burn',
 'Monster': 'Monster',
 'Burn utgått': 'Burn utgått',
 'Tonic': 'Tonic',
 'Lollipop': 'Lollipop',
 'Rett-i-koppen': 'Rett-i-koppen',
 'Battery No Cal': 'Battery No Cal',
 'Solo Boks': 'Solo Boks',
 'Pepsi Max': 'Pepsi Max',
 'Solrik': 'Solrik',
 'Farris Naturell': 'Farris Naturell',
 'Pringles': 'Pringles',
 'Battery Fresh': 'Battery Fresh',
 'Slush uten alkohol': 'Slush uten alkohol',
 'Munkholm': 'Munkholm',
 'Energidrikk': 'Energidrikk',
 'Iste': 'Iste',
 'Powerade': 'Powerade',
 'Capri Sonne': 'Capri Sonne'}

In [64]:
transaksjoner.stockName = transaksjoner.stockName.map(updated_stock_dict)
stock_sold = transaksjoner.stockName.value_counts()
stock_sold

Red Bull              4515
Brus 0,5              1488
Olden Eple            1289
Red Bull Sugarfree    1200
Brus 0,3              1113
                      ... 
Sprite 0,33              5
Red Bull 0,3             5
Lollipop                 3
Burn utgått              2
Burn 0,33                1
Name: stockName, Length: 63, dtype: int64

Siden vi kun endret en ville det gått fortere å kun bytte ut den ene

In [152]:
transaksjoner.stockName = transaksjoner.stockName.replace("Brus", "Brus 0,5")

In [65]:
stock_sold[:10]

Red Bull               4515
Brus 0,5               1488
Olden Eple             1289
Red Bull Sugarfree     1200
Brus 0,3               1113
Sjokolade               690
Coca Cola Zero 0,33     653
Monster hvit 0,5        468
Julebrus                452
Coca Cola Zero 0,5      444
Name: stockName, dtype: int64

## Julebrus

Det selges en del julebrus la oss se nærmere på det

In [72]:
julebrus = transaksjoner.loc[transaksjoner.stockName == "Julebrus"]
julebrus.head(2)

,id,userId,stockId,groupId,semesterId,logTimestamp,price,stockName,volume,groupName,mnd,semester
379,26,6,10,1,1,2022-03-23 19:52:06,15.0,Julebrus,0.33,it.gruppen,3,vår
380,51,6,10,1,1,2022-03-24 16:46:33,15.0,Julebrus,0.33,it.gruppen,3,vår


In [73]:
julebrus.groupby(["semester"]).stockName.count()

semester
høst    411
vår      41
Name: stockName, dtype: int64

In [74]:
julebrus.price.unique()

array([15., 16., 25., 20., 17.])

In [75]:
julebrus.groupby(["semester"]).price.unique()

semester
høst    [15.0, 16.0, 25.0, 20.0, 17.0]
vår           [15.0, 25.0, 20.0, 16.0]
Name: price, dtype: object

In [76]:
julebrus.groupby(["semester"]).price.value_counts()

semester  price
høst      15.0     236
          16.0     112
          17.0      44
          20.0      16
          25.0       3
vår       15.0      20
          25.0      17
          16.0       3
          20.0       1
Name: price, dtype: int64

In [81]:
pd.crosstab(julebrus.price, julebrus.semester)

semester,høst,vår
price,,
15.0,236,20
16.0,112,3
17.0,44,0
20.0,16,1
25.0,3,17


In [125]:
pd.crosstab(julebrus.price, julebrus.semester, normalize="columns")

semester,høst,vår
price,,
15.0,0.574209,0.487805
16.0,0.272506,0.073171
17.0,0.107056,0.000000
20.0,0.038929,0.024390
25.0,0.007299,0.414634


In [126]:
julebrus_pris_semester = pd.crosstab(julebrus.price, julebrus.semester, normalize="columns")
julebrus_pris_semester.style.format("{:.0%}").format_index("kr {:.0f}")

semester,høst,vår
price,,
kr 15,57%,49%
kr 16,27%,7%
kr 17,11%,0%
kr 20,4%,2%
kr 25,1%,41%


## Saving DataFrame

In [138]:
transaksjoner.to_csv("../data/mer_utfyllende_transaksjon.csv", index=False)